# Lab 1: Korean-English Translation model Training

## Introduction
---

본 모듈에서는 허깅페이스 트랜스포머(Hugging Face transformers) 라이브러리를 사용하여 한영 번역 모델을 훈련합니다. 번역은 시퀀스-투-시퀀스(sequence-to-sequence) 태스크의 가장 대표적인 형태로, 어텐션 메커니즘과 트랜스포머 기반 언어 모델의 기반이 되었던 다운스트림 태스크입니다.


### References

- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- Translation fine-tuning: https://huggingface.co/docs/transformers/tasks/translation
- KDE4 dataset: https://huggingface.co/datasets/kde4
- 관련 논문: http://www.lrec-conf.org/proceedings/lrec2012/pdf/463_Paper.pdf

In [1]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.



## 1. Setup Environments
---

### Import modules

In [2]:
import os
import sys
import json
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Trainer, TrainingArguments, set_seed
)
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers.trainer_utils import get_last_checkpoint
from datasets import load_dataset, load_metric, ClassLabel, Sequence

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Argument parser

In [3]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=32)
    parser.add_argument("--max_length", type=int, default=128)
    parser.add_argument("--stride", type=int, default=32)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=3e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--debug", type=bool, default=False)      
    parser.add_argument("--tokenizer_id", type=str, default='Helsinki-NLP/opus-mt-ko-en')
    parser.add_argument("--model_id", type=str, default='Helsinki-NLP/opus-mt-ko-en')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

In [4]:
train_dir = 'seq2seq_translate_train'
valid_dir = 'seq2seq_translate_valid'
!rm -rf {train_dir} {valid_dir}
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 `cd scripts & python3 train.py` 커맨드로 훈련 스크립트를 실행할 수 있습니다.

In [5]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'
num_gpus = torch.cuda.device_count()

!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(num_gpus)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'

args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

[{204499775.py:21} INFO - ***** Arguments *****
[{204499775.py:22} INFO - epochs=1
seed=42
train_batch_size=32
eval_batch_size=32
max_length=128
stride=32
warmup_steps=100
logging_steps=100
learning_rate=3e-05
disable_tqdm=False
fp16=True
debug=False
tokenizer_id=Helsinki-NLP/opus-mt-ko-en
model_id=Helsinki-NLP/opus-mt-ko-en
output_data_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/translation/data
model_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/translation/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/translation/seq2seq_translate_train
valid_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/translation/seq2seq_translate_valid
chkpt_dir=chkpt



<br>

## 2. Preparation & Custructing Feature set
---

### Dataset

본 핸즈온에서 사용할 데이터셋은 KDE4 데이터셋으로 한국어를 포함한 100여가지에 육박하는 언어를 지원하고 있습니다. 이 데이터셋을 사용하여, 대규모 Opus 데이터셋 (https://opus.nlpl.eu/) 으로 사전 훈련된 한영 번역 Marian 모델을 파인튜닝합니다.


- KDE4 dataset: https://huggingface.co/datasets/kde4
- 관련 논문: http://www.lrec-conf.org/proceedings/lrec2012/pdf/463_Paper.pdf

```
{
  'id': '15',
  'translation': 
    {
      'en': '& kde; provides a highly configurable desktop environment. This overview assumes that you are using the default environment.',
      'ko': '& kde; 는 다양한 부분을 설정할 수 있는 데스크톱 환경입니다. 이 문서에서는 여러분이 기본적인 데스크톱 환경을 사용한다는 것을 가정합니다.'
   }
}
````  

In [6]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="ko")

[{builder.py:463} WARNING - Using custom data configuration en-ko-lang1=en,lang2=ko
[{builder.py:641} WARNING - Reusing dataset kde4 (/home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [8]:
show_random_elements(raw_datasets["train"])

,id,translation
0,12899,"{'en': 'Archive deleted.', 'ko': '압축 파일을 삭제했습니다.'}"
1,11403,"{'en': 'Two Terminals, Horizontally', 'ko': '두 개의 터미널, 수평@ action'}"
2,52907,"{'en': 'Pacific/ Fakaofo', 'ko': '태평양/ 파카오포'}"
3,50684,"{'en': 'brown1', 'ko': 'color'}"
4,8330,"{'en': 'Replace selection', 'ko': '선택부분 바꾸기'}"
5,40103,"{'en': 'Central Region', 'ko': 'Central RegionRegion/ state in Russia'}"
6,52832,"{'en': 'Europe/ Kaliningrad', 'ko': '유럽/ 칼리닌그라드'}"
7,35057,"{'en': 'Border/ Coast', 'ko': 'ukraine. kgm'}"
8,8446,"{'en': 'Public Domain', 'ko': 'Public Domain'}"
9,46288,"{'en': 'Sami (Northern, Sweden)', 'ko': '북부 사미어 (스웨덴)'}"


In [9]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=42)
split_datasets["validation"] = split_datasets.pop("test")

[{arrow_dataset.py:3615} WARNING - Loading cached split indices for dataset at /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-7c7521c035973fea.arrow and /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-d6acd5fd405706c8.arrow


### Tokenization
데이터셋을 토큰화합니다. 원문과 타겟 번역문 모두 토큰화가 필요하며, 타겟 번역문은 context manager 내에서 `as_target_tokenizer()`로 래핑해야 합니다.

토큰화에 대한 자세한 내용은 https://huggingface.co/docs/datasets/process#processing-data-with-map 를 참조하세요.

In [10]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_id, return_tensors="pt")

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


#### Tokenize Sample Data

In [11]:
ko_sentence = split_datasets["train"][10]["translation"]["ko"]
en_sentence = split_datasets["train"][10]["translation"]["en"]

inputs = tokenizer(ko_sentence)

# If you forget to tokenize the target within the context manager, the target is tokenized by the input tokenizer. 
with tokenizer.as_target_tokenizer():
    targets = tokenizer(en_sentence)
    
inputs, targets   

({'input_ids': [20993, 343, 1068, 2266, 0], 'attention_mask': [1, 1, 1, 1, 1]},
 {'input_ids': [1097, 38774, 15193, 46, 0], 'attention_mask': [1, 1, 1, 1, 1]})

In [12]:
max_input_length = 128
max_target_length = 128

# tokenizer helper function
def preprocess_function(examples, source_lang="ko", target_lang="en"):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets;
    # If you forget to tokenize the target within the context manager, the target is tokenized by the input tokenizer. 
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

[{arrow_dataset.py:2597} WARNING - Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-2149d75340913593.arrow
[{arrow_dataset.py:2597} WARNING - Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-02f755fd9fd8c249.arrow


In [13]:
# train_dir = 'datasets/train'
# valid_dir = 'datasets/valid'
# !rm -rf {train_dir} {valid_dir}

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(valid_dir, exist_ok=True) 

train_dataset = tokenized_datasets['train']
valid_dataset = tokenized_datasets['validation']

if not os.listdir(args.train_dir):
    train_dataset.save_to_disk(args.train_dir)
if not os.listdir(args.valid_dir):
    valid_dataset.save_to_disk(args.valid_dir)

# from datasets import load_from_disk
# train_dataset = load_from_disk(args.train_dir)
# valid_dataset = load_from_disk(args.valid_dir)

<br>

## 3. Training (Fine-tuning)
---

### Define Custom metric
특정 시점마다(예: epoch, steps) 검증 데이터셋으로 정밀도(precision), 재현율(recall), F1 스코어, 정확도(accuracy)를 등의 지표를 계산하기 위한 커스텀 함수를 정의합니다.

커스텀 함수의 첫번째 인자는 `EvalPrediction` 객체로, 예측값(`predictios`)과 정답값(`label_ids`)를 포함합니다. 자세한 내용은 아래 웹사이트를 참조하세요. 
https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction

### BLEU (Bilingual Evaluation Understudy) metric
BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하는 대표적인 지표이며, 생성된 문장의 토큰이 정답(레이블) 문장에 포함되는 정도를 정량화한 n-gram 기반 precision 지표입니다.

BLEU 스코어는 0-100 스케일로 높을 수록 좋으며, 정답 레이블 대비 동일한 토큰이 계속 반복되거나 정답 레이블보다 짧은 문장을 출력 시, 페널티를 부과합니다. 

- SacreBLEU: https://github.com/mjpost/sacreBLEU

In [14]:
metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

### Pre-trained model

이제 훈련에 필요한 피쳐셋이 모두 준비되었으므로, 사전 훈련된 모델을 로드하여 파인튜닝을 수행합니다.

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(args.model_id)

### Data Collation

동적 길이의 입력 데이터를 처리하기 위해 보통 패딩(padding) 기법을 사용하며, 이 때, 허깅페이스에서 지원하는 데이터 콜레이터(Data Collator)를 사용하면 편리합니다. 번역 모델은 Seq2seq 기반으로 `DataCollatorForSeq2Seq`을 사용하면 됩니다.

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### Check sample data

In [17]:
tokenized_datasets['train'][0]

{'input_ids': [33361, 66, 0],
 'attention_mask': [1, 1, 1],
 'labels': [31874, 66, 0]}

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(0, 2)])
batch['input_ids'], batch['labels']

(tensor([[33361,    66,     0, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
          65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000],
         [11616,  9768,  1160,   182,   168,   802,   103,   956, 11616, 11964,
              9, 49537,  1160,   132,   248,     2,  8548,   438,     2,     0]]),
 tensor([[31874,    66,     0,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100],
         [21961, 17769,  2092,  2406,    13, 17769,  1911,  9159,     2, 56436,
             24,     2,     0]]))

### Training Preparation

Seq2Seq 기반 모델 평가 시에는 `predict_with_generate=True`로 설정하는 것을 잊지 마세요.

참조: https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments

In [19]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=args.chkpt_dir,
    overwrite_output_dir=True if get_last_checkpoint(args.chkpt_dir) is not None else False,    
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    weight_decay=0.01,
    learning_rate=float(args.learning_rate),    
    save_total_limit=3,
    predict_with_generate=True,
    fp16=args.fp16,
    disable_tqdm=args.disable_tqdm, 
    evaluation_strategy="no",    
    save_strategy="epoch",    
)

In [20]:
# For debug only
args.debug = True
if args.debug:
    train_dataset = train_dataset.shuffle(seed=42).select(range(3000))
    valid_dataset = valid_dataset.shuffle(seed=42).select(range(300))

[{arrow_dataset.py:3399} WARNING - Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-ae608e7ad10939de.arrow
[{arrow_dataset.py:3399} WARNING - Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/kde4/en-ko-lang1=en,lang2=ko/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac/cache-8c284c0eda7cf936.arrow


In [21]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


### Training
훈련을 수행합니다. 딥러닝 기반 자연어 처리 모델 훈련에는 GPU가 필수이며, 본격적인 훈련을 위해서는 멀티 GPU 및 분산 훈련을 권장합니다. 만약 멀티 GPU가 장착되어 있다면 Trainer에서 총 배치 크기 = 배치 크기 x GPU 개수로 지정한 다음 데이터 병렬화를 자동으로 수행합니다.

In [22]:
%%time
# train model
if get_last_checkpoint(args.chkpt_dir) is not None:
    logger.info("***** Continue Training *****")
    last_checkpoint = get_last_checkpoint(args.chkpt_dir)
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 24
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


Saving model checkpoint to chkpt/checkpoint-24
Configuration saved in chkpt/checkpoint-24/config.json
Model weights saved in chkpt/checkpoint-24/pytorch_model.bin
tokenizer config file saved in chkpt/checkpoint-24/tokenizer_config.json
Special tokens file saved in chkpt/checkpoint-24/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 24.8 s, sys: 6.46 s, total: 31.3 s
Wall time: 19.9 s


<br>

## 4. Evaluation
---

평가를 수행합니다.

In [23]:
outputs = trainer.predict(valid_dataset)
eval_results = outputs.metrics

# writes eval result to file which can be accessed later in s3 ouput
with open(os.path.join(args.output_data_dir, "eval_results.txt"), "w") as writer:
    print(f"***** Evaluation results at {args.output_data_dir} *****")
    for key, value in sorted(eval_results.items()):
        writer.write(f"{key} = {value}\n")
        logger.info(f"{key} = {value}\n")

***** Running Prediction *****
  Num examples = 300
  Batch size = 128
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Evaluation results at /home/ec2-user/SageMaker/sm-kornlp-usecases/translation/data *****
[{4005816495.py:9} INFO - test_bleu = 43.00990584009949

[{4005816495.py:9} INFO - test_loss = 1.6864901781082153

[{4005816495.py:9} INFO - test_runtime = 14.0713

[{4005816495.py:9} INFO - test_samples_per_second = 21.32

[{4005816495.py:9} INFO - test_steps_per_second = 0.213



<br>

## 5. Prediction
---

여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [24]:
from transformers import pipeline
translator = pipeline(
    task="translation",
    model=model, 
    tokenizer=tokenizer,
    device=0
)

In [25]:
translator("머신 러닝 완전 관리형 서비스인 Amazon SageMaker를 통해 쉽고 빠르게 ML모델을 개발하세요")

[{'translation_text': "It's easy and fast to develop an ML model through the Amazon SageMaker, a fully managed service."}]